<a href="https://colab.research.google.com/github/shinobu357/TugasMLRaisya/blob/main/Week%2010/Week_10_Raisya_Athaya_Kamilah_101032380253_MLP_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

# Hubungkan ke Google Drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
# Install pytorch (if not installed yet)
!pip install torch torchvision torchaudio


In [ ]:


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split



In [ ]:
# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/UTS ML/car.data.csv', header=None)

# Check the first few rows of the dataset
data.head()

,0,1,2,3,4,5,6
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [ ]:
# Map the categorical data to numeric values using LabelEncoder
label_encoder = LabelEncoder()

# Encode all features and target variable
data_encoded = data.apply(label_encoder.fit_transform)

# Split features and target variable
X = data_encoded.iloc[:, :-1].values
y = data_encoded.iloc[:, -1].values

# Split into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to torch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader
batch_size = 512
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
def train_model(model, criterion, optimizer, train_loader, epochs):
    train_losses = []
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_loss = running_loss / len(train_loader)
        train_losses.append(avg_loss)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")
    return train_losses

def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy


# RelU

In [ ]:
class MLP_ReLU(nn.Module):
    def __init__(self, input_dim, hidden_layers):
        super(MLP_ReLU, self).__init__()
        layers = []
        prev_dim = input_dim
        for h_dim in hidden_layers:
            layers.append(nn.Linear(prev_dim, h_dim))
            layers.append(nn.ReLU())  # ReLU activation
            prev_dim = h_dim
        layers.append(nn.Linear(prev_dim, len(np.unique(y))))  # Output layer with number of classes
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# Experiment with ReLU activation
hidden_layers = [4]  # Example: hidden layer with 64 neurons
epochs = 1
lr = 10

model = MLP_ReLU(input_dim=X_train.shape[1], hidden_layers=hidden_layers)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr)

# Train model
train_losses_relu = train_model(model, criterion, optimizer, train_loader, epochs)

# Evaluate model
accuracy_relu = evaluate_model(model, test_loader)


# Sigmoid

In [ ]:
class MLP_Sigmoid(nn.Module):
    def __init__(self, input_dim, hidden_layers):
        super(MLP_Sigmoid, self).__init__()
        layers = []
        prev_dim = input_dim
        for h_dim in hidden_layers:
            layers.append(nn.Linear(prev_dim, h_dim))
            layers.append(nn.Sigmoid())  # Sigmoid activation
            prev_dim = h_dim
        layers.append(nn.Linear(prev_dim, len(np.unique(y))))  # Output layer with number of classes
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# Experiment with Sigmoid activation
hidden_layers = [16,32,64]  # Example: hidden layer with 64 neurons
epochs = 250
lr = 10

model = MLP_Sigmoid(input_dim=X_train.shape[1], hidden_layers=hidden_layers)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr)

# Train model
train_losses_sigmoid = train_model(model, criterion, optimizer, train_loader, epochs)

# Evaluate model
accuracy_sigmoid = evaluate_model(model, test_loader)



Epoch [1/250], Loss: 7.4652
Epoch [2/250], Loss: 1.7456
Epoch [3/250], Loss: 1.3777
Epoch [4/250], Loss: 1.3198
Epoch [5/250], Loss: 0.8410
Epoch [6/250], Loss: 1.8773
Epoch [7/250], Loss: 1.2164
Epoch [8/250], Loss: 1.3265
Epoch [9/250], Loss: 1.2371
Epoch [10/250], Loss: 1.3836
Epoch [11/250], Loss: 0.8461
Epoch [12/250], Loss: 0.9726
Epoch [13/250], Loss: 1.3026
Epoch [14/250], Loss: 1.1855
Epoch [15/250], Loss: 1.4308
Epoch [16/250], Loss: 0.9010
Epoch [17/250], Loss: 1.2436
Epoch [18/250], Loss: 1.1755
Epoch [19/250], Loss: 1.0696
Epoch [20/250], Loss: 0.8376
Epoch [21/250], Loss: 1.6050
Epoch [22/250], Loss: 1.3875
Epoch [23/250], Loss: 1.1176
Epoch [24/250], Loss: 1.4855
Epoch [25/250], Loss: 1.4959
Epoch [26/250], Loss: 0.8429
Epoch [27/250], Loss: 1.1574
Epoch [28/250], Loss: 1.1012
Epoch [29/250], Loss: 1.0906
Epoch [30/250], Loss: 1.4517
Epoch [31/250], Loss: 1.3237
Epoch [32/250], Loss: 0.9751
Epoch [33/250], Loss: 1.4789
Epoch [34/250], Loss: 1.4872
Epoch [35/250], Loss: 1

# Tanh

In [ ]:
class MLP_Tanh(nn.Module):
    def __init__(self, input_dim, hidden_layers):
        super(MLP_Tanh, self).__init__()
        layers = []
        prev_dim = input_dim
        for h_dim in hidden_layers:
            layers.append(nn.Linear(prev_dim, h_dim))
            layers.append(nn.Tanh())  # Tanh activation
            prev_dim = h_dim
        layers.append(nn.Linear(prev_dim, len(np.unique(y))))  # Output layer with number of classes
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# Experiment with Tanh activation
hidden_layers = [16,32,64]  # Example: hidden layer with 64 neurons
epochs = 250
lr = 10

model = MLP_Tanh(input_dim=X_train.shape[1], hidden_layers=hidden_layers)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr)

# Train model
train_losses_tanh = train_model(model, criterion, optimizer, train_loader, epochs)

# Evaluate model
accuracy_tanh = evaluate_model(model, test_loader)



Epoch [1/250], Loss: 11.8059
Epoch [2/250], Loss: 79.0421
Epoch [3/250], Loss: 153.4555
Epoch [4/250], Loss: 108.4655
Epoch [5/250], Loss: 164.0088
Epoch [6/250], Loss: 172.1033
Epoch [7/250], Loss: 145.5839
Epoch [8/250], Loss: 116.9109
Epoch [9/250], Loss: 186.4050
Epoch [10/250], Loss: 153.6872
Epoch [11/250], Loss: 111.8042
Epoch [12/250], Loss: 178.1324
Epoch [13/250], Loss: 87.2985
Epoch [14/250], Loss: 189.0258
Epoch [15/250], Loss: 181.1360
Epoch [16/250], Loss: 143.3685
Epoch [17/250], Loss: 163.4900
Epoch [18/250], Loss: 164.2583
Epoch [19/250], Loss: 140.8490
Epoch [20/250], Loss: 136.0762
Epoch [21/250], Loss: 157.7492
Epoch [22/250], Loss: 102.0767
Epoch [23/250], Loss: 118.5884
Epoch [24/250], Loss: 144.7599
Epoch [25/250], Loss: 123.9152
Epoch [26/250], Loss: 160.5291
Epoch [27/250], Loss: 135.3695
Epoch [28/250], Loss: 172.6226
Epoch [29/250], Loss: 173.5915
Epoch [30/250], Loss: 146.0839
Epoch [31/250], Loss: 110.8574
Epoch [32/250], Loss: 157.2813
Epoch [33/250], Loss

# Linear

In [ ]:
class MLP_Linear(nn.Module):
    def __init__(self, input_dim, hidden_layers):
        super(MLP_Linear, self).__init__()
        layers = []
        prev_dim = input_dim
        for h_dim in hidden_layers:
            layers.append(nn.Linear(prev_dim, h_dim))
            layers.append(nn.Identity())  # Linear (no activation)
            prev_dim = h_dim
        layers.append(nn.Linear(prev_dim, len(np.unique(y))))  # Output layer with number of classes
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# Experiment with Linear activation
hidden_layers = [16,32,64]  # Example: hidden layer with 64 neurons
epochs = 250
lr = 10

model = MLP_Linear(input_dim=X_train.shape[1], hidden_layers=hidden_layers)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr)

# Train model
train_losses_linear = train_model(model, criterion, optimizer, train_loader, epochs)

# Evaluate model
accuracy_linear = evaluate_model(model, test_loader)



Epoch [1/250], Loss: 181821909728.4973
Epoch [2/250], Loss: nan
Epoch [3/250], Loss: nan
Epoch [4/250], Loss: nan
Epoch [5/250], Loss: nan
Epoch [6/250], Loss: nan
Epoch [7/250], Loss: nan
Epoch [8/250], Loss: nan
Epoch [9/250], Loss: nan
Epoch [10/250], Loss: nan
Epoch [11/250], Loss: nan
Epoch [12/250], Loss: nan
Epoch [13/250], Loss: nan
Epoch [14/250], Loss: nan
Epoch [15/250], Loss: nan
Epoch [16/250], Loss: nan
Epoch [17/250], Loss: nan
Epoch [18/250], Loss: nan
Epoch [19/250], Loss: nan
Epoch [20/250], Loss: nan
Epoch [21/250], Loss: nan
Epoch [22/250], Loss: nan
Epoch [23/250], Loss: nan
Epoch [24/250], Loss: nan
Epoch [25/250], Loss: nan
Epoch [26/250], Loss: nan
Epoch [27/250], Loss: nan
Epoch [28/250], Loss: nan
Epoch [29/250], Loss: nan
Epoch [30/250], Loss: nan
Epoch [31/250], Loss: nan
Epoch [32/250], Loss: nan
Epoch [33/250], Loss: nan
Epoch [34/250], Loss: nan
Epoch [35/250], Loss: nan
Epoch [36/250], Loss: nan
Epoch [37/250], Loss: nan
Epoch [38/250], Loss: nan
Epoch [

# Softmax

In [ ]:
class MLP_Softmax(nn.Module):
    def __init__(self, input_dim, hidden_layers):
        super(MLP_Softmax, self).__init__()
        layers = []
        prev_dim = input_dim
        for h_dim in hidden_layers:
            layers.append(nn.Linear(prev_dim, h_dim))
            layers.append(nn.ReLU())  # ReLU activation in hidden layers
            prev_dim = h_dim
        layers.append(nn.Linear(prev_dim, len(np.unique(y))))  # Output layer with number of classes
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return nn.Softmax(dim=1)(self.model(x))  # Apply Softmax on output

# Experiment with Softmax activation
hidden_layers = [16,32,64]  # Example: hidden layer with 64 neurons
epochs = 250
lr = 10

model = MLP_Softmax(input_dim=X_train.shape[1], hidden_layers=hidden_layers)
criterion = nn.CrossEntropyLoss()  # Corrected here
optimizer = optim.SGD(model.parameters(), lr=lr)

# Train model
train_losses_softmax = train_model(model, criterion, optimizer, train_loader, epochs)

# Evaluate model
accuracy_softmax = evaluate_model(model, test_loader)



Epoch [1/250], Loss: 1.1809
Epoch [2/250], Loss: 1.0364
Epoch [3/250], Loss: 1.0355
Epoch [4/250], Loss: 1.0378
Epoch [5/250], Loss: 1.0375
Epoch [6/250], Loss: 1.0341
Epoch [7/250], Loss: 1.0383
Epoch [8/250], Loss: 1.0355
Epoch [9/250], Loss: 1.0367
Epoch [10/250], Loss: 1.0392
Epoch [11/250], Loss: 1.0364
Epoch [12/250], Loss: 1.0403
Epoch [13/250], Loss: 1.0389
Epoch [14/250], Loss: 1.0389
Epoch [15/250], Loss: 1.0378
Epoch [16/250], Loss: 1.0336
Epoch [17/250], Loss: 1.0383
Epoch [18/250], Loss: 1.0397
Epoch [19/250], Loss: 1.0411
Epoch [20/250], Loss: 1.0378
Epoch [21/250], Loss: 1.0395
Epoch [22/250], Loss: 1.0403
Epoch [23/250], Loss: 1.0414
Epoch [24/250], Loss: 1.0397
Epoch [25/250], Loss: 1.0364
Epoch [26/250], Loss: 1.0372
Epoch [27/250], Loss: 1.0375
Epoch [28/250], Loss: 1.0347
Epoch [29/250], Loss: 1.0437
Epoch [30/250], Loss: 1.0350
Epoch [31/250], Loss: 1.0395
Epoch [32/250], Loss: 1.0341
Epoch [33/250], Loss: 1.0395
Epoch [34/250], Loss: 1.0431
Epoch [35/250], Loss: 1